In [1]:
!pip install pypdf2==2.0.0 -q
!pip install camelot-py==0.10.1 -q
!pip install ghostscript==0.7 -q

In [ ]:
# !pip install "camelot-py[base]"

In [9]:
import camelot
import pandas as pd

tables = camelot.read_pdf("/kaggle/input/pdf-table/table-pdf-2.pdf", pages="all")

dataframes_list = []

for i, table in enumerate(tables):
    df = table.df
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    df = df.astype(str)
    dataframes_list.append(df)
    file_name = f"table_{i + 1}.csv"
    df.to_csv(file_name, index=False)

2023-10-30T14:38:42 - INFO - Processing page-1
2023-10-30T14:38:44 - INFO - Processing page-2
2023-10-30T14:38:46 - INFO - Processing page-3
2023-10-30T14:38:48 - INFO - Processing page-4
2023-10-30T14:38:50 - INFO - Processing page-5
2023-10-30T14:38:52 - INFO - Processing page-6
2023-10-30T14:38:55 - INFO - Processing page-7
2023-10-30T14:38:58 - INFO - Processing page-8
2023-10-30T14:38:59 - INFO - Processing page-9


In [14]:
import os

ls = list(os.listdir())
dfs = []
for l in ls:
    if l[-3:] == 'csv':
        dfs.append(str(l))

In [ ]:
ls

In [15]:
concatenated_df = pd.DataFrame()
for d in range(len(dfs)):
    df = pd.read_csv(dfs[d])
    concatenated_df = pd.concat([concatenated_df, df], axis=1)

In [16]:
concatenated_df = concatenated_df.astype(str)

In [13]:
from transformers import pipeline
pipe = pipeline("table-question-answering", model="google/tapas-large-finetuned-wtq")

In [ ]:
def question_answer_pipe():
    question = input("Question:")
    answer = pipe(query=question, table=concatenated_df)['answer']
    print(f'Answer: {answer}')

In [ ]:
question_answer_pipe()

In [ ]:
'''
QUESTOINS TO ASK:

Where is Powai located?
Vihar lake is in which state?
Which is the primary inflow river in Powai lake?

'''

# API

In [2]:
!pip install -U flask-cors -q
!pip install flask -q
!pip install pyngrok -q

In [3]:
from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok
from flask_cors import CORS
from transformers import pipeline
import camelot
import pandas as pd
import os
from decouple import config

In [ ]:
app = Flask(__name__)
CORS(app)
ngrok.set_auth_token(config('AUT_TOKEN'))
public_url = ngrok.connect(8002).public_url


@app.route("/upload", methods=['POST'])
def upload():
    global concatenated_df
    file = request.files["file"]
    print('recieved')
#     os.makedirs('/kaggle/working/uploads', exist_ok=True)
    file.save("/kaggle/working/file.pdf")
    print('saved!')
    tables = camelot.read_pdf("/kaggle/working/file.pdf", pages="all")

    dataframes_list = []

    for i, table in enumerate(tables):
        df = table.df
        new_header = df.iloc[0]
        df = df[1:]
        df.columns = new_header
        df = df.astype(str)
        dataframes_list.append(df)
        file_name = f"table_{i + 1}.csv"
#         os.makedirs('/kaggle/working/csvs', exist_ok=True)
        df.to_csv("file_name.txt", index=False)
    ls = list(os.listdir())
    dfs = []
    for l in ls:
        if l[-3:] == 'csv':
            dfs.append(str(l))
    
    print('dfs made')
    
    concatenated_df = pd.DataFrame()
    print('made new df')
    for d in range(len(dfs)):
        df = pd.read_csv(dfs[d])
        concatenated_df = pd.concat([concatenated_df, df], axis=1)
    print('concat done')
        
    concatenated_df = concatenated_df.astype(str)
    print('type changed to str')
    return 'file saved'
    
@app.route("/answer", methods=['POST'])
def answer():    
    global concatenated_df
    question = request.form["text"]
    print('question recieved')
    pipe = pipeline("table-question-answering", model="google/tapas-large-finetuned-wtq")
    answer = pipe(query=question, table=concatenated_df)['answer']
    print("answer sent")
    
    return jsonify({"ans": answer})

@app.route("/delete", methods=['POST'])
def delete():
    def delete_files_in_directory(directory_path):
        try:
            files = os.listdir(directory_path)
            for file in files:
                file_path = os.path.join(directory_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("All files deleted successfully.")
        except OSError:
            print("Error occurred while deleting files.")
        os.remove('/kaggle/working/')
        return 'DOC Deleted'
    
print(f"To access the Gloable link please click {public_url}")
app.run(port=8002)

In [7]:
files = os.listdir('/')
for file in files:
    file_path = os.path.join('/', file)
    if os.path.isfile(file_path):
        os.remove(file_path)